# Demo 1 — Experiment Tracking

Train an ARIMA model on Jena daily temperature data and log everything to MLflow:
- **Parameters:** ARIMA order, seasonal order
- **Metrics:** RMSE, MAE, AIC, BIC
- **Artifacts:** forecast plot
- **Model:** saved as `arima_model` via `mlflow.statsmodels`

In [11]:
import os
import tempfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import mlflow
import mlflow.statsmodels

In [12]:
mlflow.set_tracking_uri("http://localhost:5050")
mlflow.set_experiment("temperature-forecast-simple")

2026/02/23 21:43:30 INFO mlflow.tracking.fluent: Experiment with name 'temperature-forecast-simple' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1771879410391, experiment_id='3', last_update_time=1771879410391, lifecycle_stage='active', name='temperature-forecast-simple', tags={}, workspace='default'>

## Load data & train/test split

In [13]:
df = pd.read_csv("../data/jena_daily_temp.csv", parse_dates=["Date Time"], index_col="Date Time")

# Last 90 days as test set
train = df.iloc[:-90]
test = df.iloc[-90:]

print(f"Train: {len(train)} days | Test: {len(test)} days")

Train: 2831 days | Test: 90 days


## Train ARIMA & log to MLflow

In [ ]:
order = (5, 1, 0)

with mlflow.start_run(run_name="arima-manual"):
    # Fit model
    model = ARIMA(train["temperature"], order=order)
    results = model.fit()

    # Forecast on test period
    forecast = results.forecast(steps=len(test))

    # Metrics
    rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))
    mae = mean_absolute_error(test["temperature"], forecast)

    # Log parameters
    mlflow.log_param("order", order)
    mlflow.log_param("n_train_days", len(train))
    mlflow.log_param("n_test_days", len(test))

    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("aic", results.aic)
    mlflow.log_metric("bic", results.bic)

    # Log forecast plot
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(test.index, test["temperature"], label="Actual", color="black")
    ax.plot(test.index, forecast, label="Forecast", color="red", linestyle="--")
    ax.set_title(f"ARIMA{order} — 90-day Forecast")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    plt.tight_layout()
    fig.savefig("forecast.png", dpi=100)
    mlflow.log_artifact("forecast.png")
    plt.show()

    # Log model (simple + robust across MLflow versions)
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_dir = os.path.join(tmp_dir, "arima_model")
        mlflow.statsmodels.save_model(results, path=model_dir)
        mlflow.log_artifacts(model_dir, artifact_path="arima_model")

    print(f"RMSE: {rmse:.2f} | MAE: {mae:.2f} | AIC: {results.aic:.2f}")

## Bonus: autolog

With `mlflow.statsmodels.autolog()`, statsmodels parameters and metrics are logged automatically.

In [15]:
mlflow.statsmodels.autolog()

with mlflow.start_run(run_name="arima-autolog"):
    model = ARIMA(train["temperature"], order=(3, 1, 2))
    results = model.fit()
    print(f"AIC: {results.aic:.2f}")

mlflow.statsmodels.autolog(disable=True)

/Users/daniele/miniconda3/envs/ml-gravio/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/daniele/miniconda3/envs/ml-gravio/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/daniele/miniconda3/envs/ml-gravio/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
2026/02/23 21:43:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


AIC: 12771.78
🏃 View run arima-autolog at: http://localhost:5050/#/experiments/3/runs/c2b8837d32ba4db1b7747f0d17befb7f
🧪 View experiment at: http://localhost:5050/#/experiments/3


---
**Next:** Open the MLflow UI at http://localhost:5050 and explore the logged runs.